Multiclass classification of images of numbers without using convolutional layers

In [ ]:
# download data

import torchvision

# training data
trainset = torchvision.datasets.MNIST(root='./data/', train=True, download=True)

# testing data
testset = torchvision.datasets.MNIST(root='./data/', train=False, download=True)

In [ ]:
import numpy as np

import torch
import torch.nn as nn

from torch.optim import SGD
from torch.utils.data import DataLoader

from matplotlib import pyplot as plt

import warnings 

warnings.filterwarnings('ignore')

In [ ]:
x_train = np.array([np.array(image).flatten()/255 for image, number in trainset])

y_train = np.array([number for image, number in trainset])


In [ ]:
x_test = np.array([np.array(image).flatten()/255 for image, number in testset])

y_test = np.array([number for image, number in testset])


In [ ]:
x_train = torch.from_numpy(x_train).type(torch.FloatTensor)
y_train = torch.from_numpy(y_train).type(torch.LongTensor) 

x_test = torch.from_numpy(x_test).type(torch.FloatTensor)
y_test = torch.from_numpy(y_test).type(torch.LongTensor)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

x_train = x_train.to(device)
y_train = y_train.to(device)

x_test = x_test.to(device)
y_test = y_test.to(device)

training_set = list(zip(x_train, y_train))             
test_set = list(zip(x_test, y_test))     

In [ ]:
input_size = 784
output_size = 10

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear(input_size, 100)
        self.layer2 = nn.Linear(100, 60)
        self.layer3 = nn.Linear(60, 40)
        self.layer4 = nn.Linear(40, output_size)
        self.relu = nn.ReLU()
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
    
        x = self.relu(self.layer1(x))
        x = self.relu(self.layer2(x))
        x = self.relu(self.layer3(x))
        x = self.softmax(self.layer4(x))
        return x

In [ ]:
# ----------------------------------------------------------------------------------------------------------------
#    initialize model, loss_function, optimizer & dataloader

batch_size = 16

epochs = 20
learning_rate = 0.1

model = NeuralNetwork(input_size, output_size)
model = model.to(device)                                  # GPU


optimizer = SGD(model.parameters(), lr = learning_rate)
loss_function = torch.nn.CrossEntropyLoss()              

train_dataloader = DataLoader(training_set,                 
                              batch_size = batch_size,       
                              shuffle=True)


# ----------------------------------------------------------------------------------------------------------------
#    train



batch_train_losses = []

epoch_train_losses = []
epoch_evaluation_losses = []
epoch_evaluation_accuracies = []

for epoch in range(epochs):
    
    model.train()
    
    for batch in train_dataloader:
        
        y_true = batch[1]
        input_features = batch[0]
        
        y_pred=model(input_features)
        loss=loss_function(y_pred, y_true)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        batch_loss = loss.item()
        batch_train_losses.append(batch_loss)
    
    epoch_average_loss = np.average(batch_train_losses[-len(train_dataloader):])
    epoch_train_losses.append(epoch_average_loss)

# ----------------------------------------------------------------------------------------------------------------
#   evaluation

    model.eval()
    
    y_true = y_test
    input_features = x_test
    
    y_pred = model(input_features)
    loss = loss_function(y_pred, y_true)
    
    evaluation_loss = loss.item()
    epoch_evaluation_losses.append(evaluation_loss)

# ----------------------------------------------------------------------------------------------------------------
#   test accuracy


    _, predicted_class = torch.max(y_pred, dim=1)
    accuracy = (predicted_class == y_true).float().mean().item()
    epoch_evaluation_accuracies.append(accuracy)

# ----------------------------------------------------------------------------------------------------------------
#   plot results

plt.plot(epoch_train_losses, label = 'train loss')
plt.plot(epoch_evaluation_losses, label = 'test loss', color='orange')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend()

plt.show()

plt.plot(epoch_evaluation_accuracies, label = 'test accuracy', color='orange')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend()

plt.show()
print(epoch_evaluation_accuracies[-1])